In [1]:
from pathlib import Path
import yaml
with open("config.yml", 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
path_fname = cfg['path_fname']
fname = cfg['fname']
output_dir = cfg['output_dir']

In [2]:
# Lod chunks from json file
import json
with open("Documents\\SORA_chunks_cleaned_manual.json", 'r', encoding='utf-8') as f:
    chunks = json.load(f)
print(f"Number of chunks: {len(chunks)}")

from PreProcessing.embeddingToolsFAISSv2 import EmbeddingToolFAISS
embedder = EmbeddingToolFAISS( output_dir=Path(output_dir), index_backend="faiss")
embeddings = embedder.load_index()

from RAG.ragv2 import RAG
rag_system = RAG(embedding_tool=embedder, chunks=chunks, default_mode="hybrid", reranker="colbert")

from LLM.LLM_openAI_Classification import LLMIndicatorAssistant, InitialOperationInput, IndicatorName
engine = LLMIndicatorAssistant(rag_system=rag_system)

Number of chunks: 120


c:\Users\immo\.conda\envs\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Loaded FAISS index with 120 vectors from PreProcessing\ProcessedFiles\index\faiss.index
   • Loaded metadata for 120 documents from PreProcessing\ProcessedFiles\index\docs.json


In [3]:
from typing import Dict

op = InitialOperationInput(
    maximum_takeoff_mass_category="lt_25kg",  # lt_25kg or gte_25kg
    vlos_or_bvlos="VLOS", # Visual Line of Sight VLOS or Beyond Visual Line of Sight BVLOS
    ground_environment="sparsely_populated", # "controlled_area", "sparsely_populated", "populated"
    airspace_type="uncontrolled", # "controlled", "uncontrolled"
    maximum_altitude_category="gt_50m_le_120m",  # "le_50m", "gt_50m_le_120m", "gt_120m_le_150m", "gt_150m"
)

indicators = [
    "likely_regulatory_pathway",
    "initial_ground_risk_orientation",
    "initial_air_risk_orientation",
    "expected_assessment_depth",
]

results: Dict[str, dict] = {}

for indicator in indicators:
    out = engine.answer_indicator(
        indicator=indicator,
        op=op,
        stream=False,
        print_sources=False,
    )
    results[indicator] = out["result"]


In [4]:
print("Indicators:")
for k, v in results.items():
    print(k, v)

Indicators:
likely_regulatory_pathway {'name': 'likely_regulatory_pathway', 'value': 'Specific PDRA', 'explanation': 'The operation matches the criteria of PDRA‑S01: maximum take‑off mass <25\u202fkg, visual line of sight, sparsely populated ground area, uncontrolled airspace with low risk of encountering manned aircraft, and altitude 50–120\u202fm (≤150\u202fm). Therefore the most likely regulatory pathway is a Specific PDRA.'}
initial_ground_risk_orientation {'name': 'initial_ground_risk_orientation', 'value': 'low', 'explanation': 'The operation involves a UAS with maximum take‑off mass <25\u202fkg, implying a small characteristic dimension and kinetic energy below the 34\u202fkJ threshold for the smallest size category. According to the SORA intrinsic ground risk table (see context [21]), visual line‑of‑sight operations over a sparsely populated area for the smallest category receive an intrinsic ground risk class of 2, which corresponds to a low initial ground risk orientation. Th

In [5]:
# output_path = "initial_indicators.json"
# with open(output_path, "w", encoding="utf-8") as f:
#     json.dump(results, f, indent=2, ensure_ascii=False)